# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Google Interview Question

In [1]:
import time

In [2]:
from os import urandom
from hashlib import sha1
from random import shuffle, choice

# Algorithm

In [3]:
%%time
puzzle_size = 2 ** 16

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


In [4]:
%%time
def merkles_puzzle():
    secrets = [None] * puzzle_size
    puzzles = [None] * puzzle_size

    for i in range(puzzle_size):
        # generate secret
        secrets[i] = urandom(16)

        # pair := secret|index
        pair = secrets[i] + int.to_bytes(i, 4, 'big')
        # plaintext := pair|sha1(pair)
        plaintext = pair + sha1(pair).digest()

        # cipthertext := ENCRYPT(plaintext, key)
        key = urandom(10)
        noise = sha1(key).digest()
        noise += sha1(noise).digest()
        ciphertext = bytes(i ^ j for i, j in zip(plaintext, noise))

        # puzzle := ciphertext|key
        puzzles[i] = ciphertext + key[2:]

    # randomize order
    shuffle(puzzles)

    # return
    return secrets, puzzles

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


In [5]:
%%time
def solve_puzzle(puzzle):
    ciphertext = puzzle[:40]
    key = puzzle[40:]

    for i in range(puzzle_size):
        # guess key
        noise = sha1(int.to_bytes(i, 2, 'big') + key).digest()
        noise += sha1(noise).digest()

        # plaintext := DECRYPT(ciphertext, key)
        plaintext = bytes(i ^ j for i, j in zip(ciphertext, noise))

        # pair|digest := key|index|sha1(pair)
        pair = plaintext[:20]
        digest = plaintext[20:]

        # on match: time, key, index
        if sha1(pair).digest() == digest:
            return i, pair[:16], int.from_bytes(pair[16:], 'big')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 17.6 µs


# Run

## (I) Alice

In [6]:
%%time
alice_secrets, public_puzzles = merkles_puzzle()

CPU times: user 765 ms, sys: 80 ms, total: 845 ms
Wall time: 865 ms


## (II) Bob

In [7]:
%%time
bob_time, bob_secret, public_index = solve_puzzle(choice(public_puzzles))

print('Bob has secret and publishes index')
print('key:', bob_secret)
print('index:', public_index)
print('steps executed:', bob_time)

Bob has secret and publishes index
key: b'P\xa6[\x16V6\x9a\x0f\x80\r\xd1\x9f\x1e[V\xa1'
index: 4974
steps executed: 13271
CPU times: user 109 ms, sys: 0 ns, total: 109 ms
Wall time: 109 ms


## (III) Alice

In [8]:
%%time
print('Alice has secret')
print('key:', alice_secrets[public_index])

Alice has secret
key: b'P\xa6[\x16V6\x9a\x0f\x80\r\xd1\x9f\x1e[V\xa1'
CPU times: user 144 µs, sys: 24 µs, total: 168 µs
Wall time: 162 µs


## (IV) Adversary

In [9]:
%%time
total_time, total_puzzles = 0, 0

for puzzle in public_puzzles:
    adv_time, adv_key, adv_index = solve_puzzle(puzzle)
    total_time += adv_time
    total_puzzles += 1

    if adv_index == public_index:
        print('very unlikely! adversary found secret:', adv_key)
        break

    if total_time > bob_time * 100:
        print('adversary failed to find secret')
        break

print('searched puzzles:', total_puzzles, 'steps executed:', total_time)

adversary failed to find secret
searched puzzles: 45 steps executed: 1349275
CPU times: user 12.8 s, sys: 21.1 ms, total: 12.8 s
Wall time: 13.9 s


# The End